Задача:

Есть входной текст (строка). Заданы начальный отступ (indent) и ширина выходного текста (width).

Нужно написать функцию, которая выдает текст заданной ширины. Если последняя строка короче, чем width, нужно между словами в ней добавить пробелов.

In [ ]:
def wrap_text(text:str, indent:int, width:int):
    text = " " * indent + text
    chunks, chunk_size = len(text), width
    split_lines_width = [text[i:i+width] for i in range(0, chunks, chunk_size)]
    return split_lines_width

In [ ]:
input_text = 'Однажды весною, в час небывало жаркого заката, в Москве, на Патриарших прудах, появились два гражданина. Первый из них, одетый в летнюю серенькую пару, был маленького роста, упитан, лыс, свою приличную шляпу пирожком нес в руке, а на хорошо выбритом лице его помещались сверхъестественных размеров очки в черной роговой оправе. Второй – плечистый, рыжеватый, вихрастый молодой человек в заломленной на затылок клетчатой кепке – был в ковбойке, жеваных белых брюках и в черных тапочках.'

In [ ]:
def wrap_text2(text:str, indent:int, width:int):
    wrapped_lines = [[''] * indent, ]
    split_spaces = text.split()
    for token in split_spaces:
        if len(' '.join(wrapped_lines[-1])) + len(token) < width:
            wrapped_lines[-1].append(token)
        else:
            wrapped_lines.append([token, ])

    for i in range(len(wrapped_lines)):
        wrapped_lines[i] = ' '.join(wrapped_lines[i])
        last_space_position = 0
        while len(wrapped_lines[i]) < width and ' ' in wrapped_lines[i]:
            last_space_position = wrapped_lines[i].find(' ', last_space_position+2) if wrapped_lines[i].find(' ', last_space_position+2)!=-1 else last_space_position
            wrapped_lines[i] = wrapped_lines[i][:last_space_position] + ' ' + wrapped_lines[i][last_space_position:]
        if len(wrapped_lines[i]) < width and ' ' not in wrapped_lines[i]:
            wrapped_lines[i] += ' ' * (width - len(wrapped_lines[i]))
    return wrapped_lines

In [ ]:
for line in wrap_text2(input_text, indent=5, width=25):
    print(len(line), line)

25         Однажды весною, в
25 час  небывало     жаркого
25 заката,  в  Москве,    на
25 Патриарших        прудах,
25 появились два гражданина.
25 Первый  из  них, одетый в
25 летнюю  серенькую   пару,
25 был  маленького    роста,
25 упитан,  лыс,        свою
25 приличную  шляпу пирожком
25 нес  в  руке, а на хорошо
25 выбритом  лице        его
25 помещались               
25 сверхъестественных       
25 размеров  очки  в  черной
25 роговой  оправе. Второй –
25 плечистый,     рыжеватый,
25 вихрастый молодой человек
25 в заломленной  на затылок
25 клетчатой  кепке  – был в
25 ковбойке,  жеваных  белых
25 брюках  и  в       черных
25 тапочках.                


In [ ]:
import functools

In [ ]:
vowels = [ch for ch in 'АЕЁИОУЫЭЮЯаеёиоуыэюя']
consonants = [ch for ch in 'БВГДЖЗКЛМНПРСТФХЦЧШЩбвгджзклмнпрстфхцчшщ']
BLACK_LIST = {'не', }

In [ ]:
def is_syllabic(seq):
    return not set(seq).isdisjoint(vowels)

In [ ]:
def hyphenate_positions(word):
    if word.isupper():
        return word
    return [i for i in range(1, len(word)-2)
            if is_syllabic(word[:i+1]) and
            is_syllabic(word[i+1:]) and word[:i+1].lower() not in BLACK_LIST and
           not (word[i] in consonants and word[i+1] in vowels) and
           word[i+1] not in 'ЫыЙйЪъЬь-' and
           not (word[i] in consonants and word[i]==word[i-1]) and
           not (word[i+1] in consonants and word[i+2]==word[i+1])]

In [ ]:
def hyphenate(word):
    result = []
    def do_split(prev_pos, pos):
        result.append(word[prev_pos:pos+1])
        return pos + 1
    lastpos = functools.reduce(do_split, hyphenate_positions(word), 0)
    result.append(word[lastpos:])
    return result

In [ ]:
def hyphenate_alt(word):
    return list(map(lambda pos: (word[0:pos+1], word[pos+1:]), hyphenate_positions(word)))

In [ ]:
def wrap_text_with_hyphenate(text:str, indent:int, width:int):
    wrapped_lines = [[''] * indent, ]
    split_spaces = text.split()
    token_hypenations_candidates = [hyphenate_alt(token) for token in split_spaces]
    h_flag = True
    for token, token_hypenations in zip(split_spaces, token_hypenations_candidates):
        if len(' '.join(wrapped_lines[-1])) + len(token) < width:
            wrapped_lines[-1].append(token)
        else:
            flag = True
            for variant in token_hypenations:
                for i in range(len(variant)-1):
                    if len(' '.join(wrapped_lines[-1])) + len(variant[i])+1 < width and flag and h_flag:
                        wrapped_lines[-1].append(variant[i]+'-')
                        wrapped_lines.append([variant[i+1], ])
                        flag = False
                        h_flag = False
            else:
                if flag:
                    wrapped_lines.append([token, ])
                    h_flag = True
    for i in range(len(wrapped_lines)):
        wrapped_lines[i] = ' '.join(wrapped_lines[i])
        last_space_position = 0
        while len(wrapped_lines[i]) < width and ' ' in wrapped_lines[i]:
            last_space_position = wrapped_lines[i].find(' ', last_space_position+2) if wrapped_lines[i].find(' ', last_space_position+2)!=-1 else last_space_position
            wrapped_lines[i] = wrapped_lines[i][:last_space_position] + ' ' + wrapped_lines[i][last_space_position:]
        if len(wrapped_lines[i]) < width and ' ' not in wrapped_lines[i]:
            wrapped_lines[i] += ' ' * (width - len(wrapped_lines[i]))
    return wrapped_lines

In [ ]:
for line in wrap_text_with_hyphenate(input_text, indent=4, width=30):
    print(len(line), line)

30       Однажды  весною,  в  час
30 небывало жаркого заката, в Мо-
30 скве,  на  Патриарших  прудах,
30 появились  два гражданина. Пе-
30 рвый  из  них, одетый в летнюю
30 серенькую пару, был маленького
30 роста,  упитан, лыс, свою при-
30 личную  шляпу  пирожком  нес в
30 руке, а на хорошо выбритом ли-
30 це  его             помещались
30 сверхъестественных    размеров
30 очки  в черной роговой оправе.
30 Второй – плечистый, рыжеватый,
30 вихрастый  молодой  человек  в
30 заломленной  на  затылок  кле-
30 тчатой кепке – был в ковбойке,
30 жеваных  белых  брюках и в че-
30 рных                 тапочках.


Теперь у нас есть текст, деленный на абзацы, и нам нужно разбить его на страницы.

Условия
* запрет висящих строк, т.е. первую или последнюю строку нельзя ни оставлять, ни переносить на новую страницу

* запрет разрывать абзац по строке, которая заканчивается переносом. Такая строка уходит на новую страницу.

** возможно еще какие-то условия (мы забыли)


In [ ]:
paragraphs = [
    'Однажды весною, в час небывало жаркого заката, в Москве, на Патриарших прудах, появились два гражданина.',
    'Первый из них, одетый в летнюю серенькую пару, был маленького роста, упитан, лыс, свою приличную шляпу пирожком нес в руке, а на хорошо выбритом лице его помещались сверхъестественных размеров очки в черной роговой оправе.',
    'Второй – плечистый, рыжеватый, вихрастый молодой человек в заломленной на затылок клетчатой кепке – был в ковбойке, жеваных белых брюках и в черных тапочках.']

In [ ]:
def paginate(paragraphs:list, indent:int, line_widthh:int, lines_on_page:int):
    pages = [[], ]
    paragraphs_to_lines= [wrap_text_with_hyphenate(paragraph, indent=indent, width=line_widthh) for paragraph in paragraphs]
    for paragraph_lines in paragraphs_to_lines:
        paragraph_lines_len = len(paragraph_lines)
        for i in range(paragraph_lines_len):
            if lines_on_page - len(pages[-1]) > 1:
                pages[-1].append(paragraph_lines[i])
            else:
                if paragraph_lines[i][-1] == '-':
                    pages[-1].append('FILLER')
                    pages.append([paragraph_lines[i], ])
                else:
                    if i == 1:
                        pages[-1].append('FILLER')
                        pages.append([paragraph_lines[i], ])
                    else:
                        if paragraph_lines_len - i < 1: # 1?
                            pages[-1].append('FILLER')
                            pages.append([paragraph_lines[i], ])
                        else:
                            pages[-1].append(paragraph_lines[i])
    return pages

In [ ]:
paginate(paragraphs, 3,  30, 5)

[['   Однажды весною, в час небы-',
  'вало жаркого заката, в Москве,',
  'на  Патриарших  прудах,    по-',
  'явились  два       гражданина.',
  'FILLER'],
 ['   Первый из них, одетый в ле-',
  'тнюю  серенькую  пару,     был',
  'маленького роста, упитан, лыс,',
  'свою  приличную шляпу пирожком',
  'FILLER'],
 ['нес  в  руке,  а на хорошо вы-',
  'бритом  лице  его   помещались',
  'сверхъестественных    размеров',
  'очки  в черной роговой оправе.',
  'FILLER'],
 ['    Второй  –  плечистый,  ры-',
  'жеватый,  вихрастый    молодой',
  'человек  в  заломленной на за-',
  'тылок  клетчатой кепке – был в',
  'ковбойке, жеваных белых брюках',
  'и в  черных          тапочках.']]

Алгоритм Харриса по выделению морфем из текста

In [ ]:
with open('russian (1).txt', 'r', encoding='windows-1251') as f:
    RUSSIAN_WORDS = [l.strip() for l in f.readlines()]

In [ ]:
from random import sample
from tqdm.notebook import tqdm

In [ ]:
RUSSIAN_WORDS_SAMPLE = RUSSIAN_WORDS[:5_000]

In [ ]:
RUSSIAN_WORDS_SAMPLE = ['стол', 'стола', 'столы', 'столами', 'столовая', 'столовой', 'столовыми']

In [ ]:
PREFIXES_ONE_LETTER = {}
for word in tqdm(RUSSIAN_WORDS_SAMPLE):
    word += '#'
    for i in range(1, len(word)):
        prefix = word[:i]
        variants_one_letter = set()
        for word2 in RUSSIAN_WORDS_SAMPLE:
            if word2.startswith(prefix):
                variants_one_letter.add(word[len(prefix):len(prefix)+1])
        if prefix in PREFIXES_ONE_LETTER:
            PREFIXES_ONE_LETTER[prefix] |= variants_one_letter
        else:
            PREFIXES_ONE_LETTER[prefix] = variants_one_letter

  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
PREFIXES_ONE_LETTER_to_int = [(prefix, len(vars)) for prefix, vars in PREFIXES_ONE_LETTER.items()]

In [ ]:
replace_prefix = ''
MORPHEMES = set()
for i in tqdm(range(1, len(PREFIXES_ONE_LETTER_to_int))):
    previous_prefix, previous_value = PREFIXES_ONE_LETTER_to_int[i-1]
    prefix, value = PREFIXES_ONE_LETTER_to_int[i]
    if value - previous_value > 1:
        if replace_prefix in MORPHEMES:
            MORPHEMES.add(prefix.replace(replace_prefix, ''))
            replace_prefix = ''
        else:
            replace_prefix = prefix
            MORPHEMES.add(prefix)

  0%|          | 0/8423 [00:00<?, ?it/s]

In [ ]:
print(MORPHEMES)

{'автоионн', 'абсурдны', 'автомотолотере', 'автодорожн', 'абстрагировал', 'автомодельны', 'автогенщик', 'но', 'авиамодел', 'австри', 'аберрационн', 'автогенны', 'аван', 'автоматизирует', 'абитуриент', 'авиаремонтн', 'аберрационны', 'абдукторны', 'аннотировани', 'автозаводск', 'австро', 'абсолютистск', 'автодинно', 'автоподъемник', 'абсорбиру', 'абсорбируемо', 'абордированн', 'абазинско', 'автоводителя', 'абстракционист', 'авиастроительны', 'авизны', 'Авил', 'Авдеевиче', 'автогон', 'автоматизированны', 'авантюристско', 'автобиографичн', 'авиамодельны', 'автоколонн', 'абонируемы', 'автодоени', 'авиат', 'автом', 'абразионны', 'абсурдистско', 'абсорбированны', 'авиастроитель', 'австралопитек', 'автопилот', 'абиогенн', 'щ', 'австрийско', 'автографическо', 'абаканц', 'авитаминоз', 'абстрагировавши', 'авиаопрыскивани', 'абсолютизирующи', 'авиаци', 'авизиру', 'абдукторн', 'автобиографичны', 'автомобиле', 'абелевы', 'абрам', 'Аверьян', 'авиасалон', 'авт', 'авансирую', 'автогр', 'автомати', 'авт

In [ ]:
rules = {'a': 'b', 'b': 'c', 'c': 'd', 'e': '1'}

In [ ]:
input_str = 'a,b,c,egab'

In [ ]:
def apply_rules_too_simple(input_str:str, rules:dict):
    while any(ch in rules for ch in input_str):
        for k, v in rules.items():
            input_str = input_str.replace(k, v)
    return input_str

In [ ]:
def apply_rules(input_str:str, rules:dict):
    input_str_len = len(input_str)
    input_str += ','
    while any(input_str[i] in rules
              and (i==0 and input_str[i+1] == ','
                   or input_str[i-1] == ',' and input_str[i+1] == ',')
              for i in range(1, input_str_len+1)):
        for k, v in rules.items():
            k_index = input_str.find(k)
            if (k_index!=-1 and (input_str[k_index+1]==',' and input_str[k_index-1]==',' or input_str[k_index+1]==',' and k_index==0)):
                input_str = input_str.replace(k, v, 1)
    return input_str[:-1]

In [ ]:
apply_rules_too_simple(input_str, rules)

'd,d,d,1gdd'

In [ ]:
apply_rules(input_str, rules)

'd,d,d,egab'